In [ ]:
import torch
print(torch.cuda.is_available())
# 1. Get the name of the GPU
print(f"Device: {torch.cuda.get_device_name(0)}")

# 2. Memory currently occupied by Tensors
print(f"Allocated: {torch.cuda.memory_allocated(0) / 1024**2:.2f} MB")

# 3. Memory reserved by PyTorch caching allocator (Total memory taken from OS)
print(f"Reserved: {torch.cuda.memory_reserved(0) / 1024**2:.2f} MB")

# 4. To see a full summary report
print(torch.cuda.memory_summary(device=None, abbreviated=False))

# OpenAI Whisper-large-v3-turbo

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float16

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    chunk_length_s=30,
    batch_size=1,
    dtype=torch_dtype,
    device=device,
    generate_kwargs={"language": "english", "task": "transcribe"},
    return_timestamps=True,
)

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

result = pipe(sample)
print(result["text"])

In [ ]:
import librosa
import re
def inference_turbo(path: str):
    audio, sr = librosa.load(path, sr=16000)
    with torch.no_grad():
        result = pipe(audio)
    text_clean = re.sub(r'\.', '', result['text'])
    return text_clean

In [ ]:
print(inference_turbo("data/scope_phoneme_data/A long/A long 1.wav"))

In [ ]:
import os
import pandas as pd
import gc

data_path = "data/scope_phoneme_data"
data_records = []

for folder_name in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder_name)
    
    if os.path.isdir(folder_path):
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".wav"):
                file_path = os.path.join(folder_path, file_name)
                phonemes = inference_turbo(file_path)
                
                data_records.append({
                    "Folder/Label": folder_name,
                    "File Name": file_name,
                    "Predicted Phonemes": phonemes
                })

df = pd.DataFrame(data_records)
print(df.head())

df.to_csv("results/phoneme_whisper_turbo32_results.csv", index=False)

# OpenAI Whisper-tiny

In [ ]:
print(torch.cuda.is_available())

In [ ]:
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration, pipeline
from datasets import load_dataset

# load model and processor
device = "cuda:0" if torch.cuda.is_available() else "cpu"
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
model.config.forced_decoder_ids = None


pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer, # type: ignore
    feature_extractor=processor.feature_extractor, # type: ignore
    chunk_length_s=30,
    batch_size=1,
    dtype=torch.float32,
    device=device,
    generate_kwargs={"language": "english", "task": "transcribe"},
    return_timestamps=True,
)

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

result = pipe(sample)
print(result["text"])

# # load dummy dataset and read audio files
# ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
# sample = ds[0]["audio"]
# print(type(sample["array"]), sample["array"].shape, sample["sampling_rate"])
# input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features 

# # generate token ids
# predicted_ids = model.generate(input_features)
# transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)


In [ ]:
import librosa
def inference_tiny(path: str):
    audio, sr = librosa.load(path, sr=16000)
    input_features = processor(audio, sampling_rate=sr, return_tensors="pt").input_features .to(device)
    predicted_ids = model.generate(input_features)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
    return transcription[0]

In [ ]:
import os
import pandas as pd

data_path = "data/scope_phoneme_data"
data_records = []

for folder_name in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder_name)
    
    if os.path.isdir(folder_path):
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".wav"):
                file_path = os.path.join(folder_path, file_name)
                phonemes = inference_tiny(file_path)
                
                data_records.append({
                    "Folder/Label": folder_name,
                    "File Name": file_name,
                    "Predicted Phonemes": phonemes
                })

df = pd.DataFrame(data_records)
print(df.head())

df.to_csv("results/phoneme_whisper_tiny_results.csv", index=False)